In [1]:
import os 
current_dir = os.getcwd()
print(current_dir)
os.chdir("../")

d:\interview\End-to-End-DL\research


In [2]:
from cnnClassifier.utils import read_yaml , create_directories
from cnnClassifier.constants import  *
from dataclasses import dataclass
from pathlib import Path
import os
from cnnClassifier import logging
from dataclasses import dataclass
from pathlib import Path


# @dataclass(frozen=True)
# class PrepareCallbackConfig:
#     root_dir: Path
#     base_model_path: Path
#     update_base_model_path: Path
#     params_image_size: list
#     params_learning_rate: float
#     params_include_top: bool
#     params_classes = int
#     params_weights: str


@dataclass(frozen=True)
class PrepareCallbackConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path


class ConfigurationManager:
    def __init__(self,
        config_file_path = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH):
        self.config_file = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        create_directories([self.config_file.artifacts_root])

    def get_prepare_callback_config(self) -> PrepareCallbackConfig:
        config = self.config_file.prepare_callbacks
        
        model_ckp = os.path.dirname(config.checkpoint_model_filepath)


        create_directories([Path(model_ckp) , Path(config.tensorboard_root_log_dir)])
        
        prepare_callback_config = PrepareCallbackConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )
        return prepare_callback_config

In [3]:
import tensorflow as tf
import time 
class PrepareCallback:  
    def __init__(self, config:PrepareCallbackConfig):
        self.config = config
    
    @property
    def _create_tensorboard_callback(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_log_dir = os.path.join(self.config.tensorboard_root_log_dir, f"tb_log_dir_{timestamp}")

        return tf.keras.callbacks.Tensorboard(log_dir=tb_log_dir)
    
    @property
    def _create_ckpt_callback(self):
        return  tf.keras.callbacks.ModelCheckpoint(
            filepath = self.config.checkpoint_model_filepath,
            save_best_only=True
        )

    
    def get_tb_ckpt_callback(self):
        return [
            self._create_tensorboard_callback,
            self._create_ckpt_callback
        ]


[2024-01-22 21:34:41,191: INFO: utils: Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.]
[2024-01-22 21:34:41,191: INFO: utils: NumExpr defaulting to 8 threads.]


In [4]:

try:
    config_manager = ConfigurationManager()
    base_model_config = config_manager.get_prepare_model_config()
    base_model = BaseModel(config = base_model_config)
    base_model.get_base_model()
    base_model.update_base_model()
except Exception as e:
    print(e)
    logging(e)
    

[2024-01-22 21:34:43,263: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-22 21:34:43,279: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-22 21:34:43,287: INFO: common: created directory at: artifacts]
[2024-01-22 21:34:43,290: INFO: common: created directory at: artifacts/prepare_base_model]


[2024-01-22 21:34:50,293: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                          